# Project Title
### Data Engineering Capstone Project

#### Project Summary

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession

In [2]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark
spark=create_spark_session()

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc.

We will work with immigration data to look at following trends:

- Which countries are attracting most of the immigrants
- Months/seasons for travel
- Immigrations by the visa types etc.

We are using immigration data provided in the project resources along with data related to global temperature, airports, and demographics.

In this project, we will extract the data from the files, clean the raw data and write it back to csv files. Clean files will be loaded and trasnformed to create star schema having fact and dimesion tables and write it back to parquet files. We will use spark cluster for ETL process.

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 



- **I94 Immigration Data**: This data comes from the US National Tourism and Trade Office. [This](https://www.trade.gov/national-travel-and-tourism-office) is where the data comes from. 
- **World Temperature Data**: This dataset came from Kaggle. You can read more about it [here](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data).
- **U.S. City Demographic Data**: This data comes from OpenSoft. You can read more about it [here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/).
- **Airport Code Table**: This is a simple table of airport codes and corresponding cities. It comes from [here](https://datahub.io/core/airport-codes#data).
- **ISO Codes**: The dataset contains a list of ALL the states and their codes. It can downloaded from [this Kaggle dataset]
(https://www.kaggle.com/datasets/juanumusic/countries-iso-codes)
- **Country Codes** , **Post Codes**, **Modes**, **Addr Codes**, **Visa Codes** tables are extracted from I94_SAS_Labels_Descriptions.SAS file


##### 1. Immigration Data

In [3]:
%%time
i94_df = pd.read_sas('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat', 'sas7bdat', encoding="ISO-8859-1")
i94_df.head()

CPU times: user 1min 26s, sys: 19.7 s, total: 1min 46s
Wall time: 3min 12s


In [4]:
i94_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096313 entries, 0 to 3096312
Data columns (total 28 columns):
cicid       float64
i94yr       float64
i94mon      float64
i94cit      float64
i94res      float64
i94port     object
arrdate     float64
i94mode     float64
i94addr     object
depdate     float64
i94bir      float64
i94visa     float64
count       float64
dtadfile    object
visapost    object
occup       object
entdepa     object
entdepd     object
entdepu     object
matflag     object
biryear     float64
dtaddto     object
gender      object
insnum      object
airline     object
admnum      float64
fltno       object
visatype    object
dtypes: float64(13), object(15)
memory usage: 661.4+ MB


##### 2. Global Temperature Data

In [5]:
temp_df = pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv')
temp_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [6]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
dt                               object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: float64(2), object(5)
memory usage: 459.2+ MB


##### 3. Demographics Data

In [7]:
demo_df = pd.read_csv('us-cities-demographics.csv', delimiter=';')
demo_df.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [8]:
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


##### 4. Airport Codes Data

In [9]:
airport_df = pd.read_csv('airport-codes_csv.csv')
airport_df.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [10]:
airport_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.0+ MB


#### Extract Tables from I94_SAS_Labels

In [11]:
def extract_table(start, end, sep):
    codes = []
    keys = []
    values = []
    with open('I94_SAS_Labels_Descriptions.SAS', mode='r') as file:
        for i, line in enumerate(file.readlines()):
            if i<start: pass
            elif i==end: break
            else:
                line = line.replace("'","")
                codes.append(line.strip().replace("\t",""))
    for key in codes:
        keys.append(key.split(sep)[0])
        values.append(key.split(sep)[1])
    values[-1] = values[-1].replace(" ;","")
    df = pd.DataFrame({'Code':keys, 'Name':values})
    return df

##### 5. Country Code

In [12]:
country_code_df= extract_table(9,298,' =  ')
country_code_df.to_csv('Data/Clean Data/5. country_codes.csv', index=False)
country_code_df

,Code,Name
0,582,"MEXICO Air Sea, and Not Reported (I-94, no lan..."
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA
5,324,ANGOLA
6,529,ANGUILLA
7,518,ANTIGUA-BARBUDA
8,687,ARGENTINA
9,151,ARMENIA


##### 6. Port Code

In [13]:
port_df = extract_table(302,961,'=')
port_df.to_csv('Data/Clean Data/6. port_codes.csv', index=False)
port_df

,Code,Name
0,ALC,"ALCAN, AK"
1,ANC,"ANCHORAGE, AK"
2,BAR,"BAKER AAF - BAKER ISLAND, AK"
3,DAC,"DALTONS CACHE, AK"
4,PIZ,"DEW STATION PT LAY DEW, AK"
5,DTH,"DUTCH HARBOR, AK"
6,EGL,"EAGLE, AK"
7,FRB,"FAIRBANKS, AK"
8,HOM,"HOMER, AK"
9,HYD,"HYDER, AK"


##### 7. Modes

In [86]:
modes_df = extract_table(972,976,' = ')
modes_df.to_csv('Data/Raw Data/7. modes.csv', index=False)
modes_df

,Code,Name
0,1,Air
1,2,Sea
2,3,Land
3,9,Not reported


##### 8. Addr Codes

In [15]:
addr_df = extract_table(981, 1035,'=')
addr_df.to_csv('Data/Clean Data/8. Addr_Codes.csv', index=False)
addr_df

,Code,Name
0,AL,ALABAMA
1,AK,ALASKA
2,AZ,ARIZONA
3,AR,ARKANSAS
4,CA,CALIFORNIA
5,CO,COLORADO
6,CT,CONNECTICUT
7,DE,DELAWARE
8,DC,DIST. OF COLUMBIA
9,FL,FLORIDA


##### 9. Visa Codes

In [16]:
visa_df = extract_table(1046, 1049,'= ')
visa_df.to_csv('Data/Clean Data/9. visa_codes.csv', index=False)
visa_df

,Code,Name
0,1,Business
1,2,Pleasure
2,3,Student


##### 10. ISO_Codes

In [2]:
iso_df = pd.read_csv('wikipedia-iso-country-codes.csv')
iso_df

,English short name lower case,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX
2,Albania,AL,ALB,8,ISO 3166-2:AL
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ
4,American Samoa,AS,ASM,16,ISO 3166-2:AS
5,Andorra,AD,AND,20,ISO 3166-2:AD
6,Angola,AO,AGO,24,ISO 3166-2:AO
7,Anguilla,AI,AIA,660,ISO 3166-2:AI
8,Antarctica,AQ,ATA,10,ISO 3166-2:AQ
9,Antigua and Barbuda,AG,ATG,28,ISO 3166-2:AG


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.
 - Remove rows having null values for id
 - Check if primary keys are unique
 - Check we don't have any duplicate rows

#### Cleaning Steps
Document steps necessary to clean the data

#### 1. Immigration Data

In [18]:
i94_df.columns

Index(['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
       'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count',
       'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu',
       'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline',
       'admnum', 'fltno', 'visatype'],
      dtype='object')

In [19]:
print(f'Unique i94cit: {len(i94_df.i94cit.unique())}')
print(f'Unique i94res: {len(i94_df.i94res.unique())}')

Unique i94cit: 243
Unique i94res: 229


In [20]:
i94_df.i94mode.value_counts()

1.0    2994505
3.0      66660
2.0      26349
9.0       8560
Name: i94mode, dtype: int64

In [21]:
i94_df.i94visa.value_counts()

2.0    2530868
1.0     522079
3.0      43366
Name: i94visa, dtype: int64

In [22]:
i94_df.i94cit.value_counts().head()

135.0    360157
209.0    206873
245.0    191425
111.0    188766
582.0    175781
Name: i94cit, dtype: int64

#### 2. Temperature Data

In [23]:
temp_df.columns

Index(['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City',
       'Country', 'Latitude', 'Longitude'],
      dtype='object')

In [24]:
print(f'Temparature Data contains data from {temp_df.dt.min()} to {temp_df.dt.max()}')
print(f'Total Countries = {len(temp_df.Country.unique())}')
print(f'Rows with no date = {len(temp_df[temp_df.dt.isnull()==True])}')
print(f'Rows with no AverageTemperature = {len(temp_df[temp_df.AverageTemperature.isnull()==True])}')
print(f'Rows with no Country Value = {len(temp_df[temp_df.Country.isnull()==True])}')
print(f'Average Temperature ranges from {temp_df.AverageTemperature.min()} to {temp_df.AverageTemperature.max()}')

Temparature Data contains data from 1743-11-01 to 2013-09-01
Total Countries = 159
Rows with no date = 0
Rows with no AverageTemperature = 364130
Rows with no Country Value = 0
Average Temperature ranges from -42.70399999999999 to 39.650999999999996


In [25]:
temp_df.describe()

,AverageTemperature,AverageTemperatureUncertainty
count,8.235082e+06,8.235082e+06
mean,1.672743e+01,1.028575e+00
std,1.035344e+01,1.129733e+00
min,-4.270400e+01,3.400000e-02
25%,1.029900e+01,3.370000e-01
50%,1.883100e+01,5.910000e-01
75%,2.521000e+01,1.349000e+00
max,3.965100e+01,1.539600e+01


#### 3. Demographics

In [26]:
demo_df.columns

Index(['City', 'State', 'Median Age', 'Male Population', 'Female Population',
       'Total Population', 'Number of Veterans', 'Foreign-born',
       'Average Household Size', 'State Code', 'Race', 'Count'],
      dtype='object')

In [27]:
print(f'Rows with null City = {len(demo_df[demo_df.City.isnull()==True])}')
print(f'Rows with null State = {len(demo_df[demo_df.State.isnull()==True])}')
print(f'Rows with null State Codes = {len(demo_df[demo_df["State Code"].isnull()==True])}')

Rows with null City = 0
Rows with null State = 0
Rows with null State Codes = 0


In [28]:
demo_df.describe()

,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,Count
count,2891.000000,2.888000e+03,2.888000e+03,2.891000e+03,2878.000000,2.878000e+03,2875.000000,2.891000e+03
mean,35.494881,9.732843e+04,1.017696e+05,1.989668e+05,9367.832523,4.065360e+04,2.742543,4.896377e+04
std,4.401617,2.162999e+05,2.315646e+05,4.475559e+05,13211.219924,1.557491e+05,0.433291,1.443856e+05
min,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,8.610000e+02,2.000000,9.800000e+01
25%,32.800000,3.928900e+04,4.122700e+04,8.042900e+04,3739.000000,9.224000e+03,2.430000,3.435000e+03
50%,35.300000,5.234100e+04,5.380900e+04,1.067820e+05,5397.000000,1.882200e+04,2.650000,1.378000e+04
75%,38.000000,8.664175e+04,8.960400e+04,1.752320e+05,9368.000000,3.397175e+04,2.950000,5.444700e+04
max,70.500000,4.081698e+06,4.468707e+06,8.550405e+06,156961.000000,3.212500e+06,4.980000,3.835726e+06


#### 4. Airport Codes

In [29]:
airport_df.columns

Index(['ident', 'type', 'name', 'elevation_ft', 'continent', 'iso_country',
       'iso_region', 'municipality', 'gps_code', 'iata_code', 'local_code',
       'coordinates'],
      dtype='object')

In [30]:
print(f'Rows with null ident = {len(airport_df[airport_df.ident.isnull()==True])}')
print(f'Rows with null name = {len(airport_df[airport_df.name.isnull()==True])}')
print(f'Rows with null iso_country = {len(airport_df[airport_df.iso_country.isnull()==True])}')
print(f'Rows with null continent = {len(airport_df[airport_df.continent.isnull()==True])}')

Rows with null ident = 0
Rows with null name = 0
Rows with null iso_country = 247
Rows with null continent = 27719


# Performing cleaning tasks here

- **Change Data Types**: All coumns should have approriate column types
- **Immigration**: Conversion of arrdate, depdate, and dtaddto into datetime format
- **Temparature**: Remove Rows having null AverageTemperature
- **Airport Code**: Remove Rows having null iso_country


##### 1. Immigration Data

In [31]:
i94_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096313 entries, 0 to 3096312
Data columns (total 28 columns):
cicid       float64
i94yr       float64
i94mon      float64
i94cit      float64
i94res      float64
i94port     object
arrdate     float64
i94mode     float64
i94addr     object
depdate     float64
i94bir      float64
i94visa     float64
count       float64
dtadfile    object
visapost    object
occup       object
entdepa     object
entdepd     object
entdepu     object
matflag     object
biryear     float64
dtaddto     object
gender      object
insnum      object
airline     object
admnum      float64
fltno       object
visatype    object
dtypes: float64(13), object(15)
memory usage: 661.4+ MB


In [32]:
# Change Datatypes
i94_df.i94cit = i94_df.i94cit.astype(int)
i94_df.i94res = i94_df.i94res.astype(int)
i94_df.i94yr = i94_df.i94yr.astype(int)
i94_df.i94mon = i94_df.i94mon.astype(int)
i94_df.i94mode = i94_df.i94mode.astype('category')
i94_df.i94visa = i94_df.i94visa.astype('category')

In [33]:
# Change arrdate,depdate, dtaddto, and dtaddfile to datetime
i94_df.arrdate=pd.to_timedelta(i94_df.arrdate, unit='D') + pd.Timestamp('1960-1-1')
i94_df.depdate=pd.to_timedelta(i94_df.depdate, unit='D') + pd.Timestamp('1960-1-1')
i94_df.dtaddto=pd.to_datetime(i94_df.dtaddto, format='%m%d%Y', errors = 'coerce')
i94_df.dtadfile = pd.to_datetime(i94_df.dtadfile, format='%Y%m%d')

In [34]:
i94_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096313 entries, 0 to 3096312
Data columns (total 28 columns):
cicid       float64
i94yr       int64
i94mon      int64
i94cit      int64
i94res      int64
i94port     object
arrdate     datetime64[ns]
i94mode     category
i94addr     object
depdate     datetime64[ns]
i94bir      float64
i94visa     category
count       float64
dtadfile    datetime64[ns]
visapost    object
occup       object
entdepa     object
entdepd     object
entdepu     object
matflag     object
biryear     float64
dtaddto     datetime64[ns]
gender      object
insnum      object
airline     object
admnum      float64
fltno       object
visatype    object
dtypes: category(2), datetime64[ns](4), float64(5), int64(4), object(13)
memory usage: 620.1+ MB


In [35]:
# Check missing data
i94_df.isnull().sum()

cicid             0
i94yr             0
i94mon            0
i94cit            0
i94res            0
i94port           0
arrdate           0
i94mode         239
i94addr      152372
depdate      142457
i94bir          802
i94visa           0
count             0
dtadfile          1
visapost    1881250
occup       3088187
entdepa         238
entdepd      138429
entdepu     3095921
matflag      138429
biryear         802
dtaddto       45826
gender       414269
insnum      2982605
airline       83627
admnum            0
fltno         19549
visatype          0
dtype: int64

In [38]:
i94_df[i94_df.arrdate>i94_df.depdate]

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
84805,168392.0,2016,4,582,582,HOU,2016-04-01,1.0,NaN,2016-03-31,...,NaN,M,1957.0,2016-05-07,M,NaN,UA,7.882699e+10,5574,B2
121285,244218.0,2016,4,131,131,KOA,2016-04-02,1.0,NaN,2016-03-19,...,NaN,M,1983.0,2016-06-09,F,NaN,LX,6.718949e+08,52,WT
164077,298072.0,2016,4,258,117,BOS,2016-04-02,1.0,NY,2016-01-26,...,NaN,M,1984.0,2016-04-03,M,NaN,EK,6.730104e+08,239,B1
179117,367773.0,2016,4,582,582,TUC,2016-04-02,1.0,NaN,2016-04-01,...,NaN,M,1979.0,2016-08-28,M,NaN,*GA,8.938800e+10,XAJRS,B1
182346,414662.0,2016,4,585,585,MIA,2016-04-02,1.0,NaN,2016-01-31,...,NaN,M,1977.0,2016-07-22,F,NaN,B6,6.730086e+08,924,B2
191338,425452.0,2016,4,691,691,MIA,2016-04-02,1.0,NaN,2016-04-01,...,NaN,M,1968.0,2016-10-01,M,NaN,4O,6.693548e+08,2950,B2
275275,575574.0,2016,4,582,582,MIA,2016-04-03,1.0,FL,2016-04-02,...,NaN,M,1986.0,2016-10-02,F,NaN,4O,6.724898e+08,5964,B2
294625,630663.0,2016,4,111,111,BOS,2016-04-04,1.0,NaN,2016-03-12,...,NaN,M,1995.0,NaT,F,NaN,FI,6.767670e+08,631,F1
395868,826051.0,2016,4,117,117,NOL,2016-04-05,2.0,TX,2016-04-04,...,NaN,M,1984.0,2016-06-17,M,NaN,VES,5.469735e+10,91388,WT
398137,828816.0,2016,4,124,124,HOU,2016-04-05,1.0,NaN,2016-04-04,...,NaN,M,1987.0,2016-07-03,M,NaN,UA,6.808067e+08,47,WT


In [40]:
# Remove rows having arrdate>depdate
i94_df = i94_df[i94_df.arrdate<=i94_df.depdate]
i94_df[i94_df.arrdate>i94_df.depdate]

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype


In [41]:
%%time
i94_df.to_csv('Data/Clean Data/1. i94.csv', index=False)

CPU times: user 1min 32s, sys: 742 ms, total: 1min 33s
Wall time: 1min 35s


##### 2. Global Temperature Data

In [42]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
dt                               object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: float64(2), object(5)
memory usage: 459.2+ MB


In [43]:
# Convert dt to datetime
temp_df.dt = pd.to_datetime(temp_df.dt)

In [44]:
# Remove rows with missing Avg Temperature
temp_df = temp_df[temp_df.AverageTemperature.isnull()==False]

In [45]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8235082 entries, 0 to 8599210
Data columns (total 7 columns):
dt                               datetime64[ns]
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 502.6+ MB


In [46]:
# Missing values
temp_df.isnull().sum()

dt                               0
AverageTemperature               0
AverageTemperatureUncertainty    0
City                             0
Country                          0
Latitude                         0
Longitude                        0
dtype: int64

In [47]:
temp_df.to_csv('Data/Clean Data/2. temp.csv', index=False)

#### 3. Airport Code

In [48]:
airport_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.0+ MB


In [49]:
#Remove rows missing iso_country and iso_region
airport_df = airport_df[airport_df.iso_country.isnull()==False].reset_index(drop=True)
airport_df = airport_df[airport_df.iso_region.isnull()==False].reset_index(drop=True)

In [50]:
airport_df.isnull().sum()

ident               0
type                0
name                0
elevation_ft     6990
continent       27719
iso_country         0
iso_region          0
municipality     5574
gps_code        13872
iata_code       45670
local_code      26142
coordinates         0
dtype: int64

In [51]:
airport_df.to_csv('Data/Clean Data/3. airport.csv', index=False)

#### 4. Demographics

In [52]:
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


In [53]:
# Remove Rows missing Male Population, Female Population, Number of Vetrans, Foreign-Born
demo_df = demo_df[demo_df['Male Population'].isnull()==False]
demo_df = demo_df[demo_df['Female Population'].isnull()==False]
demo_df = demo_df[demo_df['Number of Veterans'].isnull()==False]
demo_df = demo_df[demo_df['Foreign-born'].isnull()==False].reset_index(drop=True)

In [54]:
# Change datatype for Male Poulation, Female Population, Number of Vetrans, and Foreign-born
demo_df['Male Population'] = demo_df['Male Population'].astype(int)
demo_df['Female Population'] = demo_df['Female Population'].astype(int)
demo_df['Number of Veterans'] = demo_df['Number of Veterans'].astype(int)
demo_df['Foreign-born'] = demo_df['Foreign-born'].astype(int)

In [55]:
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2875 entries, 0 to 2874
Data columns (total 12 columns):
City                      2875 non-null object
State                     2875 non-null object
Median Age                2875 non-null float64
Male Population           2875 non-null int64
Female Population         2875 non-null int64
Total Population          2875 non-null int64
Number of Veterans        2875 non-null int64
Foreign-born              2875 non-null int64
Average Household Size    2875 non-null float64
State Code                2875 non-null object
Race                      2875 non-null object
Count                     2875 non-null int64
dtypes: float64(2), int64(6), object(4)
memory usage: 269.6+ KB


In [56]:
demo_df.to_csv('Data/Clean Data/4. demo.csv', index=False)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model 


 
 We will be studying the immigration data. Hence, i94 data will be used as fact table and others as dimension table as shown in the diagram.
![SCHEMA DIAGRAM](Data_Model.PNG)


#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Extract
   - Create spark dataframe for all the clean data
2. Transform
   - **fact_immigration**
       - DROP Rows with null cicid
       - DROP Rows with null gender
       - DROP Rows with NULL arrival_date
       - DROP ROWS with Invalid/should not show /No COuntry Codes as citizens or resident
       - DROP Rows with Collapsed/No port code
       - Replace citizen, resident, port, mode from the details given in I94_SAS_Labels_Descriptions.SAS
   - **dim_temperature**
       - DROP Rows with null city/country
       - Create Avg temperature field which is the average of data for all the years
   - **dim_aiport**
       - DROP Rows with Null id values
       - DROP Rows having aiport type other than small, medium and large as immigration is not possible on other airport types
   - **dim_demographics**
       - DROP Rows with no city or country data
   - **dim_time**
       - Created for all the dates in arrival_date and departure date
       
3. Load
    - Write back data models in parquet format

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

##### 1. Extract

In [3]:
# Create Spark df of all the tables
i94_spark = spark.read.format('csv').option('header', 'true').load('Data/Clean Data/1. i94.csv')
temp_spark = spark.read.format('csv').option('header', 'true').load('Data/Clean Data/2. temp.csv')
airport_spark = spark.read.format('csv').option('header', 'true').load('Data/Clean Data/3. airport.csv')
demo_spark = spark.read.format('csv').option('header', 'true').load('Data/Clean Data/4. demo.csv')
country_spark = spark.read.format('csv').option('header', 'true').load('Data/Clean Data/5. country_codes.csv')
port_spark = spark.read.format('csv').option('header', 'true').load('Data/Clean Data/6. port_codes.csv')
modes_spark = spark.read.format('csv').option('header', 'true').load('Data/Clean Data/7. modes.csv')
addr_spark = spark.read.format('csv').option('header', 'true').load('Data/Clean Data/8. Addr_Codes.csv')
visa_spark = spark.read.format('csv').option('header', 'true').load('Data/Clean Data/9. visa_codes.csv')
iso_spark = spark.read.format('csv').option('header', 'true').load('Data/Clean Data/10. wikipedia-iso-country-codes.csv')

In [4]:
i94_spark.createOrReplaceTempView("i94")
i94_spark.printSchema()

root
 |-- cicid: string (nullable = true)
 |-- i94yr: string (nullable = true)
 |-- i94mon: string (nullable = true)
 |-- i94cit: string (nullable = true)
 |-- i94res: string (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: string (nullable = true)
 |-- i94mode: string (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: string (nullable = true)
 |-- i94bir: string (nullable = true)
 |-- i94visa: string (nullable = true)
 |-- count: string (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: string (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: string (nullable = 

In [5]:
temp_spark.createOrReplaceTempView("temperature")
temp_spark.printSchema()

root
 |-- dt: string (nullable = true)
 |-- AverageTemperature: string (nullable = true)
 |-- AverageTemperatureUncertainty: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [6]:
airport_spark.createOrReplaceTempView("airport")
airport_spark.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [7]:
demo_spark.createOrReplaceTempView("demographics")
demo_spark.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: string (nullable = true)
 |-- Male Population: string (nullable = true)
 |-- Female Population: string (nullable = true)
 |-- Total Population: string (nullable = true)
 |-- Number of Veterans: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Average Household Size: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: string (nullable = true)



In [8]:
country_spark.createOrReplaceTempView("country")
port_spark.createOrReplaceTempView("ports")
modes_spark.createOrReplaceTempView("modes")
addr_spark.createOrReplaceTempView("addr")
visa_spark.createOrReplaceTempView("visa")
iso_spark.createOrReplaceTempView("iso_codes")

##### 2. Transform

In [15]:
fact_immigration = spark.sql('''
    SELECT 
        CAST(CAST(cicid AS int) AS string)   AS id,
        UPPER(c1.Name)                 AS citizen,
        UPPER(c2.Name)                 AS resident,  
        i94addr                        AS us_state,
        ports.Name                     AS port,
        NVL(modes.Name,'Not reported') AS modes,
        visa.Name                      AS visa,
        arrdate                        AS arrival_date,
        depdate                        AS departure_date,
        i94bir                         AS age,
                                          gender,
        visatype                       AS visa_type
    FROM i94
    LEFT JOIN country c1 on i94.i94cit = c1.Code
    LEFT JOIN country c2 on i94.i94res = c2.Code
    LEFT JOIN ports on i94.i94port = ports.Code
    LEFT JOIN modes on cast(i94.i94mode AS int) = modes.Code
    LEFT JOIN visa on i94.i94visa = cast(visa.Code AS float)
    WHERE cicid IS NOT NULL
    AND gender IS NOT NULL
    AND arrdate IS NOT NULL
    AND c1.Name NOT LIKE 'INVALID%'
    AND c1.Name NOT LIKE '%(should not show)'
    AND c1.Name NOT LIKE 'No Country Code%'
    AND c2.Name NOT LIKE 'INVALID%'
    AND c2.Name NOT LIKE '%(should not show)'
    AND c2.Name NOT LIKE 'No Country Code%'
    AND ports.Name NOT LIKE 'Collapsed%'
    AND ports.Name NOT LIKE 'No PORT Code%'
    
''')
fact_immigration.createOrReplaceTempView('fact_immigration')
fact_immigration.show()

+---+-------+--------+--------+-------------+-----+--------+------------+--------------+----+------+---------+
| id|citizen|resident|us_state|         port|modes|    visa|arrival_date|departure_date| age|gender|visa_type|
+---+-------+--------+--------+-------------+-----+--------+------------+--------------+----+------+---------+
| 15|ALBANIA| ALBANIA|      MI|WASHINGTON DC|  Air|Pleasure|  2016-04-01|    2016-08-25|55.0|     M|       B2|
| 27|ALBANIA| ALBANIA|      MA|   BOSTON, MA|  Air|Business|  2016-04-01|    2016-04-05|58.0|     M|       B1|
| 28|ALBANIA| ALBANIA|      MA|  ATLANTA, GA|  Air|Business|  2016-04-01|    2016-04-05|56.0|     F|       B1|
| 29|ALBANIA| ALBANIA|      MA|  ATLANTA, GA|  Air|Pleasure|  2016-04-01|    2016-04-17|62.0|     M|       B2|
| 30|ALBANIA| ALBANIA|      NJ|  ATLANTA, GA|  Air|Pleasure|  2016-04-01|    2016-05-04|49.0|     M|       B2|
| 31|ALBANIA| ALBANIA|      NY|  ATLANTA, GA|  Air|Pleasure|  2016-04-01|    2016-06-06|43.0|     M|       B2|
|

In [11]:
dim_temperature = spark.sql('''
    SELECT city, 
        UPPER(country) AS country, 
        Round(AVG(AverageTemperature),2) AS all_time_avg_temperature
    FROM temperature
    WHERE city IS NOT NULL
    AND country IS NOT NULL
    GROUP BY city, country
''')
dim_temperature.createOrReplaceTempView('dim_temperature')
dim_temperature.show()

+------------+------------------+------------------------+
|        city|           country|all_time_avg_temperature|
+------------+------------------+------------------------+
|   Allentown|     UNITED STATES|                    9.52|
|      Atyrau|        KAZAKHSTAN|                    8.07|
|     Bintulu|          MALAYSIA|                   26.16|
| Butterworth|          MALAYSIA|                   27.21|
|      Cainta|       PHILIPPINES|                   26.45|
|      Ciamis|         INDONESIA|                   24.77|
|      Dodoma|          TANZANIA|                   22.19|
|      Fuling|             CHINA|                   16.84|
|      Fuyang|             CHINA|                   15.15|
|         Ife|           NIGERIA|                   26.37|
|  Jhunjhunun|             INDIA|                   25.18|
|      Maxixe|        MOZAMBIQUE|                   24.09|
|      Owerri|           NIGERIA|                   26.61|
|Puerto Plata|DOMINICAN REPUBLIC|                   26.0

In [12]:
dim_airport = spark.sql('''
    SELECT ident AS id,
           CASE
               WHEN type='small_airport' THEN 'Small'
               WHEN type='medium_airport' THEN 'Medium'
               WHEN type='large_aiport' THEN 'Large'
            END         AS type,
           name,
           iso_country AS country,
           iso_region  AS region
    FROM airport
    WHERE ident IS NOT NULL 
    AND type IN ('small_aiport', 'medium_airport', 'large_aiport')
''')
dim_airport.createOrReplaceTempView('dim_airport')
dim_airport.show()

+-------+------+--------------------+-------+------+
|     id|  type|                name|country|region|
+-------+------+--------------------+-------+------+
|    5A8|Medium|Aleknagik / New A...|     US| US-AK|
|AE-0030|Medium|FIVE STAR FINANCE...|     AE| AE-DU|
|   AGGH|Medium|Honiara Internati...|     SB| SB-CT|
|   AGGM|Medium|       Munda Airport|     SB| SB-WE|
|    AHJ|Medium|    Hongyuan Airport|     CN| CN-51|
|AL-0004|Medium|Çá¸¾á¸á¸ á¸®á¸...|     AL|AL-U-A|
|   ANYN|Medium|Nauru Internation...|     NR| NR-14|
|AU-0118|Medium|                 LST|     AU|AU-TAS|
|AU-0120|Medium|                 Wbl|     AU|AU-VIC|
|AU-0121|Medium|                YKCY|     AU|AU-QLD|
|    AXF|Medium|Alxa Left Banner ...|     CN| CN-15|
|   AYBK|Medium|        Buka Airport|     PG|PG-NSB|
|   AYCH|Medium|      Chimbu Airport|     PG|PG-CPK|
|   AYDU|Medium|        Daru Airport|     PG|PG-WPD|
|   AYGA|Medium|      Goroka Airport|     PG|PG-EHG|
|   AYGN|Medium|      Gurney Airport|     PG|P

In [13]:
dim_demographics = spark.sql('''
    SELECT City              AS city,
    UPPER(State)             AS state,
    `Median Age`             AS median_age,
    `Male Population`        AS male_population,
    `Female Population`      AS female_population,
    `Total Population`       AS total_population,
    `Foreign-born`           AS foreign_Born,
    `Average Household Size` AS avg_household_size,
    `State Code`             AS state_code,
    Race                     AS race
    FROM demographics
    WHERE City IS NOT NULL
    AND State IS NOT NULL
''')
dim_demographics.createOrReplaceTempView('dim_demographics')
dim_demographics.show()

+----------------+--------------+----------+---------------+-----------------+----------------+------------+------------------+----------+--------------------+
|            city|         state|median_age|male_population|female_population|total_population|foreign_Born|avg_household_size|state_code|                race|
+----------------+--------------+----------+---------------+-----------------+----------------+------------+------------------+----------+--------------------+
|   Silver Spring|      MARYLAND|      33.8|          40601|            41862|           82463|       30908|               2.6|        MD|  Hispanic or Latino|
|          Quincy| MASSACHUSETTS|      41.0|          44129|            49500|           93629|       32935|              2.39|        MA|               White|
|          Hoover|       ALABAMA|      38.5|          38040|            46799|           84839|        8229|              2.58|        AL|               Asian|
|Rancho Cucamonga|    CALIFORNIA|      3

In [14]:
dim_time = spark.sql('''
    WITH date AS ( SELECT DISTINCT date FROM (
        SELECT arrdate AS date FROM i94
        UNION 
        SELECT depdate AS date FROM i94
        )
    )
    
    SELECT date,
        YEAR(date)                   AS year,
        DATE_FORMAT(date, 'MMM')     AS month,
        DAY(date)                    AS day,
        WEEKOFYEAR(date)             AS week,
        QUARTER(date)                AS quarter
    FROM date
    ORDER BY date
''')
dim_time.createOrReplaceTempView('dim_time')
dim_time.show()

+----------+----+-----+---+----+-------+
|      date|year|month|day|week|quarter|
+----------+----+-----+---+----+-------+
|2016-04-01|2016|  Apr|  1|  13|      2|
|2016-04-02|2016|  Apr|  2|  13|      2|
|2016-04-03|2016|  Apr|  3|  13|      2|
|2016-04-04|2016|  Apr|  4|  14|      2|
|2016-04-05|2016|  Apr|  5|  14|      2|
|2016-04-06|2016|  Apr|  6|  14|      2|
|2016-04-07|2016|  Apr|  7|  14|      2|
|2016-04-08|2016|  Apr|  8|  14|      2|
|2016-04-09|2016|  Apr|  9|  14|      2|
|2016-04-10|2016|  Apr| 10|  14|      2|
|2016-04-11|2016|  Apr| 11|  15|      2|
|2016-04-12|2016|  Apr| 12|  15|      2|
|2016-04-13|2016|  Apr| 13|  15|      2|
|2016-04-14|2016|  Apr| 14|  15|      2|
|2016-04-15|2016|  Apr| 15|  15|      2|
|2016-04-16|2016|  Apr| 16|  15|      2|
|2016-04-17|2016|  Apr| 17|  15|      2|
|2016-04-18|2016|  Apr| 18|  16|      2|
|2016-04-19|2016|  Apr| 19|  16|      2|
|2016-04-20|2016|  Apr| 20|  16|      2|
+----------+----+-----+---+----+-------+
only showing top

In [44]:
dim_iso = spark.sql('''
SELECT UPPER(`English short name lower case`) AS country,
       `Alpha-2 code`                         AS iso_code
FROM iso_codes
''')
dim_iso.createOrReplaceTempView('dim_iso')
dim_iso.show()

+-------------------+--------+
|            country|iso_code|
+-------------------+--------+
|        AFGHANISTAN|      AF|
|      ÅLAND ISLANDS|      AX|
|            ALBANIA|      AL|
|            ALGERIA|      DZ|
|     AMERICAN SAMOA|      AS|
|            ANDORRA|      AD|
|             ANGOLA|      AO|
|           ANGUILLA|      AI|
|         ANTARCTICA|      AQ|
|ANTIGUA AND BARBUDA|      AG|
|          ARGENTINA|      AR|
|            ARMENIA|      AM|
|              ARUBA|      AW|
|          AUSTRALIA|      AU|
|            AUSTRIA|      AT|
|         AZERBAIJAN|      AZ|
|            BAHAMAS|      BS|
|            BAHRAIN|      BH|
|         BANGLADESH|      BD|
|           BARBADOS|      BB|
+-------------------+--------+
only showing top 20 rows



#### Analysis

1. Top 5 countries having most no. of immigrants

In [17]:
immigrants_count_by_country = spark.sql('''
    SELECT citizen AS country,
    COUNT(id) AS count_of_immigrants
    FROM fact_immigration
    GROUP by citizen
    ORDER BY COUNT(id) DESC
    LIMIT 5
''')
immigrants_count_by_country.show()

+--------------------+-------------------+
|             country|count_of_immigrants|
+--------------------+-------------------+
|      UNITED KINGDOM|             277984|
|               JAPAN|             162932|
|          CHINA, PRC|             162814|
|MEXICO AIR SEA, A...|             162254|
|              FRANCE|             152989|
+--------------------+-------------------+



2. Countries with Highest Avg temperature

In [18]:
temp_by_country = spark.sql('''
    SELECT country, all_time_avg_temperature
    FROM dim_temperature
    ORDER BY all_time_avg_temperature DESC
    LIMIT 5
''')
temp_by_country.show()

+--------+------------------------+
| country|all_time_avg_temperature|
+--------+------------------------+
|DJIBOUTI|                   29.15|
|   SUDAN|                   29.08|
|   SUDAN|                   29.08|
|   NIGER|                   29.06|
|   SUDAN|                   28.94|
+--------+------------------------+



3. Departures by Month

In [19]:
immi_by_month = spark.sql('''
    SELECT t.Month, COUNT(id) FROM fact_immigration f
    LEFT JOIN dim_time t ON f.departure_date=t.date
    GROUP BY t.month
    ORDER BY count(id) DESC
''')
immi_by_month.show()

+-----+---------+
|Month|count(id)|
+-----+---------+
|  Apr|  1374668|
|  May|   692692|
|  Jun|    77314|
|  Jul|    42555|
|  Aug|    24208|
|  Sep|    13543|
|  Jan|        1|
+-----+---------+



4. Common Visa Types

In [20]:
visa = spark.sql('''
    SELECT Visa, visa_type, COUNT(id) FROM fact_immigration
    GROUP BY visa, visa_type
    ORDER BY COUNT(id) DESC
''')
visa.show()

+--------+---------+---------+
|    Visa|visa_type|count(id)|
+--------+---------+---------+
|Pleasure|       B2|   929251|
|Pleasure|       WT|   910478|
|Business|       B1|   175137|
|Business|       WB|   150524|
| Student|       F1|    24640|
|Business|       E2|    13436|
|Pleasure|       CP|    11977|
|Business|        I|     2658|
|Business|       E1|     2333|
|Pleasure|      GMT|     2205|
| Student|       F2|     1489|
| Student|       M1|      582|
|Business|       I1|      196|
|Business|      GMB|       43|
| Student|       M2|       23|
|Pleasure|      CPL|        7|
|Pleasure|      SBP|        2|
+--------+---------+---------+



5. Immigrants by gender

In [21]:
gender = spark.sql('''SELECT gender, count(id) FROM fact_immigration
    GROUP BY gender
''')
gender.show()

+------+---------+
|gender|count(id)|
+------+---------+
|     F|  1076737|
|     M|  1147633|
|     U|      224|
|     X|      387|
+------+---------+



6. Immigrants by mode of transport

In [22]:
mode = spark.sql('''SELECT modes, count(id) FROM fact_immigration
    GROUP BY modes
''')
mode.show()

+------------+---------+
|       modes|count(id)|
+------------+---------+
|         Sea|    13902|
|        Land|    51663|
|         Air|  2157194|
|Not reported|     2222|
+------------+---------+



7. Correlation of Average Temperature and Immigrants for a country

In [31]:
spark.sql('SELECT MAX(all_time_avg_temperature), MIN(all_time_avg_temperature) FROM dim_temperature').show()

+-----------------------------+-----------------------------+
|max(all_time_avg_temperature)|min(all_time_avg_temperature)|
+-----------------------------+-----------------------------+
|                        29.15|                       -11.85|
+-----------------------------+-----------------------------+



In [32]:
temp = spark.sql('''
    WITH temperature AS (
        SELECT Country, 
        ROUND(AVG(all_time_avg_temperature),2) AS all_time_avg_temperature 
        FROM dim_temperature
        GROUP BY country
    ),
    
    immigrants(
        SELECT Citizen AS Country, COUNT(id) AS no_of_immigrants
        FROM fact_immigration
        GROUP BY Citizen
    )
    
    SELECT i.Country,  no_of_immigrants, all_time_avg_temperature
    FROM immigrants i LEFT JOIN temperature t
    ON i.Country=t.Country
    WHERE all_time_avg_temperature IS NOT NULL
    ORDER BY no_of_immigrants DESC
''')
temp.show()

+--------------+----------------+------------------------+
|       Country|no_of_immigrants|all_time_avg_temperature|
+--------------+----------------+------------------------+
|UNITED KINGDOM|          277984|                     9.1|
|         JAPAN|          162932|                   13.41|
|        FRANCE|          152989|                   10.34|
|        BRAZIL|          113359|                   21.99|
|         INDIA|           83717|                   25.38|
|     AUSTRALIA|           81732|                   16.82|
|     ARGENTINA|           64327|                   16.93|
|   NETHERLANDS|           59995|                     9.1|
|         ITALY|           57728|                   12.62|
|         SPAIN|           49605|                   14.51|
|      COLOMBIA|           48496|                   22.63|
|       ECUADOR|           39988|                   20.61|
|     VENEZUELA|           37381|                   25.48|
|        SWEDEN|           36869|                    5.6

 8. Population, Avg Household size for US States with more than 50K Immigration 

In [57]:
population = spark.sql('''
    WITH immigrants AS (
    SELECT us_state, count(id) AS no_of_Immigrants
    FROM fact_immigration WHERE us_state <> '.N' 
    GROUP BY us_state
    HAVING count(id)>50000
    )
    
    SELECT state_code, state, no_of_immigrants, 
        ROUND(AVG(median_age),2) AS median_age,
        CAST(SUM(male_population) AS int) AS male_population,
        CAST(SUM(female_population) AS int) AS female_population, 
        CAST(SUM(total_population) AS int) AS total_population, 
        ROUND(AVG(avg_household_size),2) AS avg_household_size
    FROM immigrants i 
    LEFT JOIN dim_demographics d
        ON i.us_state=d.state_code
    GROUP BY state_code, no_of_immigrants, state
''')
population.show()

+----------+----------+----------------+----------+---------------+-----------------+----------------+------------------+
|state_code|     state|no_of_immigrants|median_age|male_population|female_population|total_population|avg_household_size|
+----------+----------+----------------+----------+---------------+-----------------+----------------+------------------+
|        HI|    HAWAII|          119663|      41.4|         884035|           879795|         1763830|              2.69|
|        IL|  ILLINOIS|           57601|     35.71|       10943864|         11570526|        22514390|              2.73|
|        CA|CALIFORNIA|          347628|     36.17|       61055672|         62388681|       123444353|               3.1|
|        NJ|NEW JERSEY|           56626|     35.25|        3423033|          3507991|         6931024|              2.96|
|        FL|   FLORIDA|          486451|      39.1|       15461937|         16626425|        32088362|              2.76|
|        NY|  NEW YORK| 

##### 3. Load

In [ ]:
%%time
fact_immigration.write.mode('overwrite').partitionBy('citizen', 'resident').parquet("Data/Transform Data/fact_immigration")
dim_temperature.write.mode('overwrite').partitionBy('country', 'city').parquet("Data/Transform Data/dim_temperature")
dim_airport.write.mode('overwrite').parquet("Data/Transform Data/dim_airports")
dim_demographics.write.mode('overwrite').parquet("Data/Transform Data/dim_demographics")
dim_time.write.mode('overwrite').parquet("Data/Transform Data/dim_time")

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

--------------------------------------------------------------------------------------------------------------------------
1. Check all tables have not null primary key

In [47]:
def non_null_check(spark, table_column_dict):
    flag = 0
    for table in table_column_dict:
        print(f'Non Null DQ Check for {table}')
        for column in table_column_dict[table]:
            DQ_NonNull = spark.sql(f'SELECT * FROM {table} WHERE {column} IS NULL')
            if len(DQ_NonNull.head(1))==0:
                print(f'\t- Non Null DQ check passed for {column}')
            else: 
                print(f'\t- Non Null DQ check failed for {column}')
                flag = 1
    if flag==0: print('Non Null DQ check passed for all tables')
    else: print('Non Null DQ check not passed for all tables')

In [49]:
table_column_dict = {'fact_immigration':['id', 'gender'], 
                     'dim_temperature': ['city', 'country'],
                     'dim_airport': ['id'],
                     'dim_demographics': ['city', 'state'],
                     'dim_time': ['date'],
                     'dim_iso': ['country', 'iso_code_2']}

non_null_check(spark, table_column_dict)

Non Null DQ Check for fact_immigration
	- Non Null DQ check passed for id
	- Non Null DQ check passed for gender
Non Null DQ Check for dim_temperature
	- Non Null DQ check passed for city
	- Non Null DQ check passed for country
Non Null DQ Check for dim_airport
	- Non Null DQ check passed for id
Non Null DQ Check for dim_demographics
	- Non Null DQ check passed for city
	- Non Null DQ check passed for state
Non Null DQ Check for dim_time
	- Non Null DQ check passed for date
Non Null DQ Check for dim_iso
	- Non Null DQ check passed for country
	- Non Null DQ check passed for iso_code_2
Non Null DQ check passed for all tables


In [75]:
def unique_check(spark, table_column_dict):
    flag = 0
    for table in table_column_dict:
        print(f'Unique DQ Check for {table}')
        column = ', '.join(table_column_dict[table])
        DQ_Unique = spark.sql(f'SELECT IF(COUNT({column})=COUNT(DISTINCT {column}),True,False) FROM {table}')
        if DQ_Unique.head()[0]==True:
            print(f'\t- Unique DQ check passed for {column}')
        else:
            flag = 1
            print(f'\t- Unique DQ check failed for {column}')
    if flag==0: print('Unique DQ check passed for all tables')
    else: print('Unique DQ check not passed for all tables')

In [76]:
table_column_dict = {'fact_immigration':['id'], 
                     'dim_temperature': ['city', 'country'],
                     'dim_airport': ['id'],
                     'dim_demographics': ['city', 'state', 'race'],
                     'dim_time': ['date'],
                     'dim_iso': ['country']}

unique_check(spark, table_column_dict)

Unique DQ Check for fact_immigration
	- Unique DQ check passed for id
Unique DQ Check for dim_temperature
	- Unique DQ check passed for city, country
Unique DQ Check for dim_airport
	- Unique DQ check passed for id
Unique DQ Check for dim_demographics
	- Unique DQ check passed for city, state, race
Unique DQ Check for dim_time
	- Unique DQ check passed for date
Unique DQ Check for dim_iso
	- Unique DQ check passed for country
Unique DQ check passed for all tables


------------------------------------------------------------------------------------------------------------------
2. fact_immigration:
    - Citizen or resident should not have Invalid/(should not show)/No Country Code 
    - Ports should not have Collapsed/No Port Code

In [19]:
DQ_fact_immigration = spark.sql('''
    SELECT * FROM fact_immigration
    WHERE citizen LIKE 'INVALID%'
    OR citizen LIKE '%(should not show)'
    OR citizen LIKE 'No Country Code%'
    OR resident LIKE 'INVALID%'
    OR resident LIKE '%(should not show)'
    OR resident LIKE 'No Country Code%'
    OR port LIKE 'Collapsed%'
    OR port LIKE 'No PORT Code%'
''')
if len(DQ_fact_immigration.head(1))==0:
    print('DQ Check for fact_immigration Passed!')

DQ Check for fact_immigration Passed!


------------------------------------------------------------------------------------------------------------------
3. dim_airport:
    - Airport type should be small, medium or large
    - region contains country-region. Verify if country=split(region,'-')[0]

In [22]:
DQ_dim_aiport = spark.sql('''
    SELECT * FROM dim_airport
    WHERE type NOT IN ('Small', 'Medium', 'Large')
''')
if len(DQ_dim_aiport.head(1))==0:
    print('DQ Check for dim_aiport Passed!')

DQ Check for dim_aiport Passed!


In [27]:
DQ_dim_airport2 = spark.sql('''
    SELECT * FROM dim_airport
    WHERE SPLIT(region,'-')[0] <> country
''')
if len(DQ_dim_aiport.head(1))==0:
    print('DQ Check for dim_aiport Passed!')

DQ Check for dim_aiport Passed!


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.
- Data Dictionary is present in Data Dictionary.txt

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

- Our largest table i.e. fact_immigration contains ~3m rows with airport, demographics, and temperature date. For processing this in efficient manner, spark will work well here due to its ability to handle large data
- We are looking at below analysis, and i94 is updated monthly so all other data can also be refreshed monthly:
    - Which countries are attracting most of the immigrants
    - Months/seasons for travel
    - Immigrations by the visa types, gender, travel modes etc.
    
- If data was increased by 100x, we can use Amazon EMR and Amazon Redshift for better processing and storage respectively.
- If the data populates a dashboard that must be updated on a daily basis by 7am every day, we can use Apache Airflow and create DAGs to schedule and monitor daily Data Quality checks and run ETL pipelines.
- If the database needed to be accessed by 100+ people, we can use cluster on Amzon Redshift which is scalabble and can manage access for multiple users easily.
    